In [1]:
import pymongo
cli = pymongo.MongoClient(
    host='165.227.64.16',
    ssl=True,
    ssl_ca_certs='ca.pem',
    ssl_match_hostname=False)

In [2]:
db = cli.test
db.authenticate('test', 'test')

True

# Switch to Cloud Manager

Now, we'll set up managed sharding in the Cloud Manager console

In [3]:
import time
from functools import wraps

def timed(label):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            n = kwargs.get('n')
            bs = kwargs.get('bs', 1)
            begin = time.time()
            try:
                return func(*args, **kwargs)
            finally:
                elapsed = time.time() - begin
                if n:
                    if bs:
                        n *= bs
                    print('{} {} / s'.format(
                        n / elapsed,
                        label))
                else:
                    print('{}: {}s elapsed'.format(
                        label, elapsed))
        return wrapper
    return decorator

In [4]:
coll = db.stress

In [5]:
@timed('inserts')
def insert_test(coll, n=1000, bs=1000):
    for ix_n in range(n):
        block = [
            {'ix_n': ix_n, 'ix_b': ix_b}
            for ix_b in range(bs)]
        coll.insert_many(block)
        if ix_n % 40 == 0:
            print()
        print('.', end='', flush=True)
    print()

In [6]:
import random
@timed('queries')
def query_test(coll, max_ix_b, max_ix_n, n=100, bs=10):
    for ix_n in range(n):
        rv_n = random.randint(0, max_ix_n + 1)
        rv_b = random.randint(0, max_ix_b + 1)
        res = coll.find({
            'ix_n': rv_n,
            'ix_b': {'$gt': rv_b}
        })
        list(res)


In [7]:
@timed('updates')
def update_test(coll, max_ix_b, max_ix_n, n=10000):
    for ix_n in range(n):
        rv_n = random.randint(0, max_ix_n + 1)
        rv_b = random.randint(0, max_ix_b + 1)
        coll.update_one(
            {'ix_n': rv_n, 'ix_b': rv_b},
            {'$inc': {'x': 1}})

In [8]:
insert_test(coll, n=500, bs=1000)


........................................
........................................
........................................
........................................
........................................
........................................
........................................
........................................
........................................
........................................
........................................
........................................
....................
8993.490107108135 inserts / s


In [9]:
max_ix_b = 999
max_ix_n = 499

In [10]:
coll.create_index([
    ('ix_n', 1),
    ('ix_b', 1)
])

'ix_n_1_ix_b_1'

In [ ]:
from threading import Thread
threads = [
    Thread(target=insert_test, args=(coll,), kwargs={'n': 2000, 'bs': 1000}),
    Thread(target=update_test, args=(coll,), 
           kwargs={'n': 20000, 'max_ix_n': max_ix_n, 'max_ix_b': max_ix_b}),
    Thread(target=query_test, args=(coll,), 
           kwargs={'n': 2000, 'max_ix_n': max_ix_n, 'max_ix_b': max_ix_b})]
for t in threads:
    t.setDaemon(True)
    t.start()
for t in threads:
    t.join()


........................................
........................................
........................................
........................................
........................................
..............16.541630006201892 queries / s
..........................
........................................
........................................
........................................
........................................
........................................
........................................
........................................
........................................
........................................
........................................
...........................